<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-бибилиотек-и-загрузка-данных" data-toc-modified-id="Импорт-бибилиотек-и-загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт бибилиотек и загрузка данных</a></span></li><li><span><a href="#Удаление-подписей" data-toc-modified-id="Удаление-подписей-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Удаление подписей</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Баланс-классов" data-toc-modified-id="Баланс-классов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Баланс классов</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Векторизация-текста" data-toc-modified-id="Векторизация-текста-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Векторизация текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Простейшая-модель" data-toc-modified-id="Простейшая-модель-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Простейшая модель</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Модель-Байеса" data-toc-modified-id="Модель-Байеса-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель Байеса</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт бибилиотек и загрузка данных

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.pipeline import Pipeline

import nltk

import transformers

In [2]:
try:
    data = pd.read_csv("/datasets/toxic_comments.csv")
except FileNotFoundError:
    data = pd.read_csv("toxic_comments.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
with pd.option_context("display.max_colwidth", None):
    display(data["text"].head(10))

0                                                                                                                                                                                                                                                                                                                                                                             Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
1                                                                                                                                                                                                                                                                                                                                                                               

Данные успешно загружены, можно приступать к обработке.

Тексты довольно грязные и в них есть лишние (служебные) данные - [подписи](https://en.wikipedia.org/wiki/Wikipedia:Signatures) авторов, т.е. IP, таймпстампы и пр. Их уберем регексами.

Также уберем ссылки: они не являются обычным языком, соответственно, включать их в модель не стоит.

### Удаление подписей

In [5]:
regexes = [
    r"(\d{1,3}\.){3}\d{1,3}",  ## IP адрес
    r"(\d{2}:\d{2}), (January|February|March|April|May|June|July|August|September|October|November|December) (\d{1,2}, \d{4} \(UTC\))",  ## Таймстамп
    r"https*://\S*" ## Ссылки
]

Выделим корпус, он еще понадобится в дальнейшем.

Преобразовывать будем его.

In [6]:
## На всякий случай конвертнем в Unicode
corpus = data["text"].astype("U")

In [7]:
for item in regexes:
    regex = re.compile(item)
    corpus = corpus.str.replace(regex, "")

In [8]:
with pd.option_context("display.max_colwidth", None):
    print(corpus.head(10))

0                                                                                                                                                                                                                                                                                                                                                                                         Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.
1                                                                                                                                                                                                                                                                                                                                                                               

Данные немного почистили, можно приступать к лемматизации.

### Лемматизация

Весь текст у нас на английском, так что для лемматизации будем пользоваться библиотекой NLTK.

Напишем функцию, которая будет лемматизировать и разделять исходный текст.

In [9]:
def split_and_lemm(text:str, lemmatizer) -> str:
    
    def match_tags(tag) -> str:
        ## Сопоставляет теги, возвращаемые nltk.pos_tag с аргументами .lemmatize()
        
        adj_list = ["JJ", "JJR", "JJS"]
        verb_list = ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
        adverb_list = ["RB", "RBR", "RBS"]
        
        if tag in adj_list:
            return "a"
        elif tag in verb_list:
            return "v"
        elif tag in adverb_list:
            return "r"
        else:
            return "n"
    
    
    text = nltk.pos_tag(nltk.word_tokenize(text))
    result = []
    for item in text:
        word = item[0].lower() ## слова, начинающиеся с заглавных, не лемматизируются, поэтому .lower()
        pos_tag = match_tags(item[1])
        result.append(lemmatizer.lemmatize(word, pos=pos_tag))
        
    return " ".join(result)

Функция готова, теперь проверим ее на каком-нибудь контрольном тексте.

In [10]:
test_text = "The Wheel of Time turns, and ages come and pass, leaving memories that become legend. Legends fade to myth, and even myth is long forgotten when the Age that gave it birth comes again.\n In one Age, called the third age by some, an Age yet to come, an age long pass, a wind rose in the Mountains of Mist. The wind was not the beginning. There are neither beginnings or endings to the turning of the Wheel of Time. But it was a beginning."

Функции NLTK завязаны на некоторые пакеты, поэтому загрузим их.

In [11]:
nltk.download(["wordnet", "punkt", "averaged_perceptron_tagger"])

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/metal_undivided/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/metal_undivided/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/metal_undivided/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

In [13]:
print("Original text:\n", test_text, "\n")
print("Lemmatized text:\n", split_and_lemm(test_text, lemmatizer))

Original text:
 The Wheel of Time turns, and ages come and pass, leaving memories that become legend. Legends fade to myth, and even myth is long forgotten when the Age that gave it birth comes again.
 In one Age, called the third age by some, an Age yet to come, an age long pass, a wind rose in the Mountains of Mist. The wind was not the beginning. There are neither beginnings or endings to the turning of the Wheel of Time. But it was a beginning. 

Lemmatized text:
 the wheel of time turn , and age come and pas , leave memory that become legend . legends fade to myth , and even myth be long forget when the age that give it birth come again . in one age , call the third age by some , an age yet to come , an age long pas , a wind rise in the mountain of mist . the wind be not the beginning . there be neither beginning or ending to the turning of the wheel of time . but it be a beginning .


Вроде выглядит достойно, можно пробовать применять ко всему корпусу.

In [14]:
%%time
corpus = corpus.apply(lambda item: split_and_lemm(item, lemmatizer))

CPU times: user 9min 53s, sys: 1.95 s, total: 9min 55s
Wall time: 9min 55s


In [15]:
with pd.option_context("display.max_colwidth", None):
    print(corpus.head(10))

0                                                                                                                                                                                                                                                                                                                                                                                 explanation why the edits make under my username hardcore metallica fan be revert ? they be n't vandalisms , just closure on some gas after i vote at new york doll fac . and please do n't remove the template from the talk page since i 'm retired now .
1                                                                                                                                                                                                                                                                                                                                                                                         

Функция работает весьма медленно, но корпус вроде похож на правду.

Теперь можно делить данные на обучающую и тестовую выборки и векторизовывать текст.

### Баланс классов

Прежде чем делить данные на выборки, посмотрим, как распределены классы в целевом признаке.

In [16]:
print("Доля токсичных комментариев в корпусе: {0:.2%}".format(data["toxic"].sum() / data.shape[0]))

Доля токсичных комментариев в корпусе: 10.16%


Есть сильный дисбаланс классов в целевом признаке, будем это учитывать при обучении моделей.

### Разделение на выборки

Разделим данные на обучающую и тестовую выборки, в пропорции 80/20. Подбор гиперпараметров будем осуществлять кросс-валидацией.

Предиктором у нас выступит сам векторизованный текст, целевым признаком - флаг токсичности.

In [17]:
seed = 111  ## для воспроизводимости результатов

In [18]:
features_train, features_test, target_train, target_test = train_test_split(corpus, data["toxic"],
                                                                            train_size=.8, random_state=seed)

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(127433,)
(127433,)
(31859,)
(31859,)


Данные разделили, теперь можно векторизовывать обучающую выборку.

### Векторизация текста

Векторизовывать будем через TF-IDF, это позволит присвоить больший вес менее часто встречаемым словам, которые нам и интересны.

Поскольку поиск гиперпараметров будет при помощи кросс-валидации, фит векторизатора запакуем в пайплайн и будем применять его в итерациях GridSearchCV, иначе получется утечка данных в валидационные части выборки.

In [19]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/metal_undivided/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stop_words = nltk.corpus.stopwords.words("english")

In [21]:
#vectorizer = TfidfVectorizer(stop_words=stop_words)

In [22]:
#features_train = vectorizer.fit_transform(features_train)

Векторы получены, можно пробовать обучать модели.

## Обучение

### Простейшая модель

Прежде чем обучать модели, стоит понять, какой скор можно считать адекватным.

Для этого обучим простую модель, которая предсказывает самое распространненое значение (в нашем случае все комментарии нетоксичные).

In [23]:
dummy_model = GridSearchCV(
    DummyClassifier(),
    param_grid={},
    scoring="f1"
)

In [24]:
dummy_model.fit(features_train, target_train)

GridSearchCV(estimator=DummyClassifier(), param_grid={}, scoring='f1')

In [25]:
dummy_model.best_score_

0.0

Ну в принципе, все математично и логично.

### Логистическая регрессия

Начнем с самой простой модели - логистической регрессии.

In [26]:
linear_params = {
    "model__class_weight":["balanced"],
    "model__max_iter":[200],
    "model__random_state":[seed]
}

linear_pipeline = Pipeline([
    ("vect", TfidfVectorizer(stop_words=stop_words)),
    ("model", LogisticRegression())
])

linear_model = GridSearchCV(
    linear_pipeline,
    param_grid=linear_params,
    scoring="f1",
    #verbose=3
)

In [27]:
%%time
linear_model.fit(features_train, target_train)

CPU times: user 2min 30s, sys: 5.42 s, total: 2min 36s
Wall time: 1min 8s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('model', LogisticRegression())]),
             param_grid={'model__class_weight': ['balanced'],
                         'model__max_iter': [200],
                         'model__random_state': [111]},
             scoring='f1')

In [28]:
linear_model.best_score_

0.7408291760624209

Точность немного не дотягивает до условия задания, нужно искать еще варианты.

Деревья решений здесь подойдут слабо - слишком много признаков.

То же самое и с kNN - слишком много данных, как признаков, так и наблюдений.

Еще будем вести сводную таблицу.

In [29]:
model_summary = pd.DataFrame(
    {"accuracy":linear_model.best_score_},
    index=["Logistic Regression"]
)

### Модель Байеса

Попробуем Байесовы модели, судя по описанию, они должны хорошо подходить под данную задачу.

В sklearn их несколько, в описании ComplementNB написано, что он хорошо справляется с несбалансированными данными, что мне и нужно.

Гауссова Байеса я попробовал обучить, но у меня на него не хватает памяти.

In [30]:
bayes_grid = {
    #"model__alpha":np.arange(1.5, .5, -.1)
}

bayes_pipeline = Pipeline([
    ("vect", TfidfVectorizer(stop_words=stop_words)),
    ("model", ComplementNB())
])

bayes_model = GridSearchCV(
    bayes_pipeline,
    param_grid=bayes_grid
)

In [31]:
%%time
bayes_model.fit(features_train, target_train)

CPU times: user 38.2 s, sys: 284 ms, total: 38.5 s
Wall time: 38.1 s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('model', ComplementNB())]),
             param_grid={})

In [32]:
bayes_model.best_score_

0.9347892633235715

Точность весьма достойная, на этом, думаю, можно было бы завершать поиск модели и идти к проверке на тестовой выборке, но еще хочется попробовать BERT.

In [33]:
model_summary = pd.concat([model_summary] + 
    [pd.DataFrame(
        {"accuracy":bayes_model.best_score_},
        index=["Complement Bayes"]
    )]
)

### BERT

In [35]:
tokenizer = transformers.BertTokenizer()

2023-06-01 18:55:34.083626: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 18:55:34.083740: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


RuntimeError: Failed to import transformers.models.bert because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны